In [1]:
%load_ext autoreload
%autoreload 2

# fundamentals
import os, sys
import numpy as np
import pandas as pd
from calendar import monthrange, month_name
import scipy.stats as stats
import datetime
import imp
import scipy.io as sio
import pickle as pkl

# plotting libraries and setup
from matplotlib.colors import BoundaryNorm
import matplotlib.pyplot as plt
plt.style.use('nrelplot')
from windrose import WindroseAxes

# met mast functions and utilities
sys.path.append('../')
import met_funcs as MET
import vis as vis
import utils as utils

In [2]:
datapath = '/Users/nhamilto/Documents/Wake_Dynamics/SiteChar/data/IEC_tmp/'
monthly_events_files = os.listdir(datapath)
today = datetime.date.today()
figpath = '../figs_{}{}{}'.format(str(today.year), str(today.month).zfill(2), str(today.day).zfill(2))

try:
    os.makedirs(figpath)
except:
    pass

In [3]:
def EDC_alt(sonicdat, params, T=6.0):
    # smoothsonic = sonicdat.rolling(60, min_periods=1).mean()
    tmp = MET.sonic_data_resampler(sonicdat, 6.0)

    # calculate diff: Delta_WD = WD_(t+1) - WD_(t-1)
    tmpa = tmp['WD_mean'].diff(periods=1)
    tmpb = tmp['WD_mean'].diff(periods=-1)
    tmp['deltaWD'] = tmpa-tmpb

    # Orient Delta_WD onto compass (i.e. change > 180 degrees corresponds to a change in the other direction)
    tmp.deltaWD[tmp.deltaWD > 180] = -360 + tmp.deltaWD[tmp.deltaWD > 180]
    tmp.deltaWD[tmp.deltaWD < -180] = 360 + tmp.deltaWD[tmp.deltaWD < -180]

    # Turbulence standard deviation depends on mean wind speed
    tmp['sigma_1'] = params['Iref'] * (0.75 * tmp['WS_mean'] + 5.6)

    # Direction change threshold depends on wind speed
    tmp['delta_WD_thresh'] = np.degrees(4 * np.arctan( tmp['sigma_1'] / (tmp['WS_mean'] * (1 + 0.1 * params['D'] / params['Lambda_1']))))

    # event detection
    tmpEDC = tmp[(tmp['deltaWD'] > tmp['delta_WD_thresh']) | (tmp['deltaWD'] < -tmp['delta_WD_thresh'])]
    
    return tmpEDC

In [ ]:
# time range
# years  = [ int(a) for a in np.arange(2012,2019,1) ] #
# months = [ int(a) for a in np.arange(1,12.1,1) ]
# days = [int(a) for a in np.arange(1,31.1,1)]
# 2012-10-01
years  = [ 2012 ] #
months = [ 10 ]
days = [int(a) for a in np.arange(1,31.1,1)]

# paths (must mount volume smb://nrel.gov/shared/wind/WindWeb/MetData/135mData/)
towerID = 'M5'
metDataPath = '/Volumes/135mData/{}Twr/20Hz/mat/'.format(towerID)

savepath = '/Users/nhamilto/Documents/Wake_Dynamics/SiteChar/data/IEC_tmp'
try:
    os.makedirs(savepath)
except:
    pass

wskeys = ['Cup_WS_C1_130m',
 'Cup_WS_122m',
 'Cup_WS_C1_105m',
 'Cup_WS_87m',
 'Cup_WS_C1_80m',
 'Cup_WS_C1_55m',
 'Cup_WS_38m',
 'Cup_WS_C1_30m',
 'Cup_WS_10m',
 'Cup_WS_3m',
 'Sonic_CupEqHorizSpeed_119m',
 'Sonic_CupEqHorizSpeed_100m',
 'Sonic_CupEqHorizSpeed_74m',
 'Sonic_CupEqHorizSpeed_61m',
 'Sonic_CupEqHorizSpeed_41m',
 'Sonic_CupEqHorizSpeed_15m']

wdkeys = ['Vane_WD_122m',
 'Vane_WD_87m',
 'Vane_WD_38m',
 'Vane_WD_10m',
 'Vane_WD_3m',
 'Sonic_direction_119m',
 'Sonic_direction_100m',
 'Sonic_direction_74m',
 'Sonic_direction_61m',
 'Sonic_direction_41m',
 'Sonic_direction_15m']

# %%timeit
probeheight = str(87)
datakeys = [['time_UTC'], [x for x in wskeys if probeheight in x], [x for x in wdkeys if probeheight in x]]
datakeys = [item for sublist in datakeys for item in sublist]
keys = ['time', 'WS', 'WD']
datakeys = {key:value for key,value in zip(keys,datakeys)}

# extract variables needed for classificiation of IEC events
params = MET.setup_IEC_params() # sonicdat, probeheight=100
    
for year in years:
    for month in months:

        # begin empty lists for events
        EOGevents = pd.DataFrame()
        EOGeventsALT = pd.DataFrame()
        
        print('reading 20Hz data for {}/{}'.format(year, month))

        for day in days:            
            datapath = os.path.join(metDataPath, str(year),
                                    str(month).zfill(2),
                                    str(day).zfill(2))
            
            # temp arrays for data i/o
            WS = np.array([])
            WD = np.array([])
            time = np.array([])
            mats = []
            
            try:
                filelist = os.listdir(datapath)
            except:
                print('data not found: {}'.format(datapath))
                continue
                              
            for file in filelist:
                tmp = sio.loadmat( os.path.join(datapath, file) , variable_names=datakeys.values())
                WS = np.append(WS, tmp[datakeys['WS']][0][0][0].flatten())
                WD = np.append(WD, tmp[datakeys['WD']][0][0][0].flatten())
                time = np.append(time, tmp[datakeys['time']][0][0][0].flatten())

                metdat = pd.DataFrame(index=pd.DatetimeIndex(utils.matlab_datenum_to_python_datetime(time)), 
                                      data=np.vstack((WS, WD)).T, columns=['WS', 'WD'])

            # replace 0 values with nans, 
            # drop all nan values,
            # rolling average of 3 s
                metdat = metdat.replace(to_replace=0.0, value=np.NaN).dropna(how='any').rolling(60, min_periods=1).mean()

            EOG_events_found = MET.find_EOG_events(metdat, params)
            EOGCevents = pd.concat([EOGevents, EOG_events_found])
            
            EOG_events_found = find_EOG_events_ALT(metdat, params)
            EOGCeventsALT = pd.concat([EOGevents, EOG_events_found])
            
#             EDC_events_found = MET.find_EDC_events(metdat, params)
#             EDCevents = pd.concat([EDCevents, EDC_events_found])

#             EDC_events_found = EDC_alt(metdat, params)
#             EDCeventsALT = pd.concat([EDCeventsALT, EDC_events_found])
            
#             ECD_events_found = MET.find_ECD_events(sonicdat, params)
#             ECDevents = pd.concat([ECDevents, ECD_events_found])

#                 allshearcalcs = pd.concat([allshearcalcs, shearevents])
                                
#         if len(EWSevents) > 0:
#             filename = 'EWSevents_{}_{}.pkl'.format(year, month)
#             savefile = os.path.join(savepath, filename)
#             print('EWS event detected. Stored to: {}'.format(filename))
#             with open(savefile, 'wb') as f:
#                 pkl.dump(EWSevents, f, pkl.HIGHEST_PROTOCOL)

reading 20Hz data for 2012/10


/Users/nhamilto/.local/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
/Users/nhamilto/.local/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


In [ ]:
EOGCevents.shape

In [ ]:
EOGCeventsALT.shape

In [22]:
###########################################
def find_EOG_events_ALT(sonicdat, params, T=10.5):
    """
    Find extreme operating wind gust events.

    This function takes inputs from a pandas DataFrame containing all of the desired data and International Electrotechnical Commission (IEC) parameters at the given period for search, determines extreme operating wind gust events, and returns the findings in an object which can be used to index files later.

        :param sonicdat: A pandas DataFrame containing all of the desired data at the given probe height including wind speed, wind direction, and the date and timestamps of the input data.

        :param params: A dictionary containing all of the parameters established by the IEC for the given input data and probe height.

        :param T: A float used to define the period for search (Seconds).

        :returns EOGeventfound: An object used to store any significant extreme operating wind gust events.
    """
    # resample at T seconds
    sonic_10_5s = MET.sonic_data_resampler(sonicdat, T)

    sonic_10_5s['WS_mean'] = sonic_10_5s.WS_mean.resample('10T').mean()
    sonic_10_5s['WS_mean'].interpolate('nearest')

    # calc IEC standard velocity variance
    sigma_1 = params['Iref'] * (0.75 * sonic_10_5s['WS_mean'] + 5.6)

    test1 = 1.35 * (params['Ve01'] - sonic_10_5s['WS_mean'])
    test2 = 3.3 * (sigma_1 / (1 + 0.1 * params['D'] / params['Lambda_1']))

    # IEC gust velocity magnitude threshold
    Vgust = np.min(np.vstack([test1.values, test2.values]), axis=0)

    t = np.linspace(0, T, 101)
    WS_pos_gustlim = np.zeros(Vgust.shape)
    WS_neg_gustlim = np.zeros(Vgust.shape)
    for ii, vv in enumerate(Vgust):
        mod = 0.37 * vv * np.sin(
            3 * np.pi * t / T) * (1 - np.cos(2 * np.pi * t / T))
        WS_pos_gustlim[ii] = sonic_10_5s['WS_mean'].iloc[ii] - mod.min()
        WS_neg_gustlim[ii] = sonic_10_5s['WS_mean'].iloc[ii] - mod.max()

    sonic_10_5s['WS_pos_gustlim'] = WS_pos_gustlim
    sonic_10_5s['WS_neg_gustlim'] = WS_neg_gustlim

    posmask = sonic_10_5s['WS_max'] > sonic_10_5s['WS_pos_gustlim']
    negmask = sonic_10_5s['WS_min'] < sonic_10_5s['WS_neg_gustlim']

    # test for EOG events (pos or pos+neg)
    singletest = sonic_10_5s[posmask]
    # doubletest = sonic_10_5s[posmask & negmask]

    return singletest  #, doubletest


###########################################

In [14]:
metdat['10minmean']=metdat.WS.resample('10T').mean()

In [17]:
metdat['10minmean'].interpolate('nearest').dropna(how='any').shape

(1566561,)

In [18]:
metdat.shape

(1674560, 3)